#downloading major crypto tickers

In [ ]:
!pip install requests

In [ ]:
# chatgpt: create a python list of the 100 major cryptocurrencies tickers please
import requests

def get_top_100_crypto_tickers():
    url = 'https://api.coingecko.com/api/v3/coins/markets'
    params = {
        'vs_currency': 'usd',
        'order': 'market_cap_desc',
        'per_page': 100,
        'page': 1,
        'sparkline': 'false'
    }

    response = requests.get(url, params=params)
    response.raise_for_status()  # Raises HTTPError for bad responses

    data = response.json()
    tickers = [coin['symbol'].upper() for coin in data]
    return tickers

# Example usage
tickers = get_top_100_crypto_tickers()

for i in range (len(tickers)):
  tickers[i] += '-USD'
print(tickers)

# stable coins removed using chatgpt

#Downloading data

In [ ]:
import yfinance as yf
import pandas as pd

In [ ]:
tickers_crypto =  ['BTC-USD', 'ETH-USD', 'XRP-USD', 'BNB-USD', 'SOL-USD', 'DOGE-USD', 'ADA-USD', 'TRX-USD', 'STETH-USD', 'WBTC-USD',
                   'LINK-USD', 'AVAX-USD', 'XLM-USD', 'LEO-USD', 'HBAR-USD', 'WSTETH-USD', 'BCH-USD', 'LTC-USD', 'DOT-USD', 'BSC-USD-USD',
                   'BGB-USD', 'WETH-USD', 'XMR-USD', 'WBT-USD', 'WEETH-USD', 'CBBTC-USD', 'PEPE-USD', 'TAO-USD', 'NEAR-USD', 'OKB-USD',
                   'ONDO-USD', 'GT-USD', 'ICP-USD', 'ETC-USD', 'AAVE-USD', 'KAS-USD', 'TKX-USD', 'CRO-USD', 'VET-USD', 'RENDER-USD',
                   'ATOM-USD', 'ALGO-USD', 'LBTC-USD', 'FET-USD', 'ENA-USD', 'FTN-USD', 'FIL-USD', 'BONK-USD', 'JLP-USD', 'SOLVBTC-USD',
                   'KCS-USD', 'OP-USD', 'MKR-USD', 'XDC-USD', 'BNSOL-USD', 'NEXO-USD', 'FLR-USD', 'QNT-USD', 'FARTCOIN-USD', 'SEI-USD',
                   'WETH-USD', 'IP-USD', 'IMX-USD', 'EOS-USD', 'RSETH-USD', 'INJ-USD', 'GRT-USD', 'WBNB-USD', 'VIRTUAL-USD', 'CRV-USD',
                   'RAY-USD', 'RETH-USD']

In [ ]:
folder = './datasets/crypto/hourly/'
assets = tickers_crypto
print(assets)

In [ ]:
import time
import random

prd='max' # period of aquisition
ntrv='1h' # interval of aquisition
column = 'Close'

# data = backup.copy()
#backup = data.copy()

print(f"Downloading data for {assets[0]}")
data = yf.download(assets[0], period=prd, interval = ntrv)[column]

# Loop through the remaining assets
for t in range(1, len(assets)):  # Downloading only the second asset for demonstration
    ticker = assets[t]
    try:
        print(f"\nDownloading data for {ticker} {t+1}/{len(assets)}")
        new_data = yf.download(ticker, period=prd, interval=ntrv)[column]

        # if first or last row is nan try to download again 2 more times
        cont = 0
        while (pd.isna(new_data.iloc[0,0]) or pd.isna(new_data.iloc[-1,0])) and cont < 2:
          print(f"\nDownloading data for {ticker} AGAIN {t+1}/{len(assets)}")
          wait_time = random.randint(1, 3)
          print(f"Waiting for {wait_time} seconds...")
          time.sleep(wait_time)
          new_data = yf.download(ticker, period=prd, interval=ntrv)[column]
          cont += 1

        data = pd.concat([data, new_data], axis=1, join='outer')
        data = data.rename(columns={column: ticker})
        display(data)
        wait_time = random.randint(1, 3)
        print(f"Waiting for {wait_time} seconds...")
        time.sleep(wait_time)
    except Exception as e:
        print(f"Error downloading data for {ticker}: {e}")
        #break

data

In [ ]:
# prompt: drop df columns with all nan values

# Identify columns where the first row is NaN
columns_to_drop = data.columns[data.iloc[0].isna()]

# Drop those columns
df = data.drop(columns=columns_to_drop)

df

In [ ]:
# prompt: add a column 'Real' in the first position of df
df.insert(0, 'Real', True)
df

In [ ]:
# adding missing rows to ensure frequency is hourly
df = df.asfreq('h')#.ffill()
# df.reset_index(inplace=True)
#df.rename(columns={'index': 'Datetime'}, inplace=True)
df.index.name = 'Datetime'
df

In [ ]:
# prompt: set Values of Real column from df to False where it's not True
df.loc[df['Real'] != True, 'Real'] = False
df[df['Real'] != True]

In [ ]:
# prompt: fill df with missing values except for 'Real' column

# Fill missing values in all columns except 'Real'
for col in df.columns:
    if col != 'Real' and col != 'Datetime':
        df[col] = df[col].fillna(method='ffill')
df

In [ ]:
# prompt: drop df columns with all nan values
df.dropna(axis=1, inplace=True)
df

In [ ]:
# prompt: drop the following columns tickers from df: [ USDT-USD,USDC-USD, SUI-USD, TON-USD, SHIB-USD, HYPE-USD, USDE-USD, PI-USD, APT-USD, UNI-USD, DAI-USD, TRUMP-USD, MNT-USD, POL-USD, TIA-USD, ARB-USD, WLD-USD, JUP-USD, STX-USD, USDC-USD ]

tickers_to_drop = ['USDT-USD', 'USDC-USD', 'SUI-USD', 'TON-USD', 'SHIB-USD', 'HYPE-USD', 'USDE-USD', 'PI-USD', 'APT-USD', 'UNI-USD', 'DAI-USD', 'TRUMP-USD', 'MNT-USD', 'POL-USD', 'TIA-USD', 'ARB-USD', 'WLD-USD', 'JUP-USD', 'STX-USD', 'USDC-USD']

# Check if the tickers exist in the DataFrame before dropping
columns_to_drop = [ticker for ticker in tickers_to_drop if ticker in df.columns]

df = df.drop(columns=columns_to_drop)


In [ ]:
# prompt: plot all values of df from column 2 and beyond
# investigate and exclude assets with strange behaviour (ex: constant value for long time, stable coins, etc.)

import matplotlib.pyplot as plt

# Assuming 'df' is already defined from the previous code

for col in df.columns[1:]:  # Iterate through columns starting from the second one
    plt.figure(figsize=(10, 6))  # Adjust figure size if needed
    plt.plot(df[col])
    plt.title(f'Plot of {col}')
    plt.xlabel(df.columns[0]) # Assuming the first column is the date/time
    plt.ylabel(col)
    plt.grid(True)
    plt.show()

In [ ]:
df

In [ ]:
# prompt: drop df columns where last row value is lower than 1

# Drop columns where the last row value is lower than 1
df = df.loc[:, df.iloc[-1] >= 1]
df

In [ ]:
def save_df(df, path):
    df2 = df.reset_index()
    df2.to_csv(path, index=False)

In [ ]:
save_df(df, 'dataset.csv')

In [ ]:
pd.read_csv('dataset.csv')